In [ ]:
import pandas as pd
import numpy as np
import findspark
findspark.init('/srv/spark')
import pyspark
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import glob


In [ ]:
sc = SparkContext()
sq = pyspark.sql.SQLContext(sc)

In [ ]:
data = pd.read_csv('data_train.csv')

In [ ]:
data.head()

In [ ]:
def cross_vad(y,ratio) :

    n = y.shape[0]
    interval = int(n *ratio)
    seed = 3
    np.random.seed(seed)
    index = np.random.permutation(n)
    k_train = index[:interval]
    k_test  = index[interval:]
    return k_train , k_test

In [ ]:
dict_ ={'User':[],'Movie':[]}
for id_ in ((data).Id.values) :
    split = id_.split('_')
    dict_['User'] += [split[0][1:]]
    dict_['Movie'] += [split[1][1:]]
formated_data = pd.DataFrame(dict_)
formated_data.head()

In [ ]:
formated_data.loc[:,'Rating'] = data.Prediction

In [ ]:
index_train, index_test = cross_vad(data,0.7)
train = formated_data.loc[index_train]
test  = formated_data.loc[index_test]
test.head()

In [ ]:
test_on = test.drop('Rating',1)

In [ ]:
pDF = sq.createDataFrame(train)
spDF = pDF.rdd

In [ ]:
# Build the recommendation model using Alternating Least Squares
lambdas = np.logspace(-10,-0.01,10)
#for lambda_ in lambdas :
rank = 6 
lambda_ = 0.07585775750291851
print('rank = {c} and lambda = {l}'.format(c=(rank),l=lambda_))
#for i in range(5,21) :

print('iterations = {c}'.format(c=(17)))
#rank = i+1
numIterations = 17
model = ALS.train(spDF, rank, numIterations,lambda_)
pDF_test = sq.createDataFrame(test_on)
spDF_test = pDF_test.rdd
testdata = spDF_test.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: (r[0], r[1], r[2]))
s  = predictions.toDF()
lol = s.toPandas()
lol.sort_values(['_1','_2'],ascending=[1,1],inplace=True)
predic_end = lol.reset_index(drop=True)
test.Movie = test.Movie.values.astype(int)
test.User = test.User.values.astype(int)
new_test = test.sort_values(by=['Movie','User'],ascending=[True,True])
test_end = new_test.reset_index(drop=True)
test_end.head()
print(np.mean((test_end.Rating.values-predic_end._3.values)**2))

In [ ]:
data_test = pd.read_csv('sampleSubmission.csv')

In [ ]:
dict_ ={'User':[],'Movie':[]}
for id_ in ((data_test).Id.values) :
    split = id_.split('_')
    dict_['User'] += [split[0][1:]]
    dict_['Movie'] += [split[1][1:]]
formated_data = pd.DataFrame(dict_)
formated_data.head()

In [ ]:
pDF = sq.createDataFrame(formated_data)
spDF = pDF.rdd

In [ ]:
testdata = spDF.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: (r[0], r[1], r[2]))


In [ ]:
s  = predictions.toDF()

In [ ]:
lol = s.toPandas()

In [ ]:
lol.sort_values(['_1','_2'],ascending=[1,1],inplace=True)
predic_end = lol.reset_index(drop=True)

In [ ]:
predic_end[predic_end._3 < 1 ] = 1
predic_end[predic_end._3 > 5 ] = 5
predic_end.head()

In [ ]:
data_test.Prediction = predic_end._3
data_test.to_csv('submission.csv',index=False)